In [1]:
setwd("/frazer01/projects/CARDIPS/analysis/cardiac_eqtls")

source("script/packages.R"  )
source("script/input_data.R")
source("script/functions.R" )


# Analysis of differential expression results

In [2]:
gene_info               = fread("pipeline/1.2.expression/gene_info.txt"   , sep = "\t", header = TRUE, data.table = FALSE)
isof_info               = fread("pipeline/1.2.expression//isoform_info.txt", sep = "\t", header = TRUE, data.table = FALSE)
gene_info$transcript_id = gene_info$gene_id

In [ ]:
read_diffexp = function(name, gene_info)
{
    indata               = fread(paste("pipeline/4.1.differential_expression", paste("diffexp_cell", name, "txt", sep = "."), sep = "/"), sep = "\t", header = TRUE, data.table = FALSE)
    indata               = merge(gene_info[,c("transcript_id", "gene_id", "gene_name", "gene_type")], indata)
    indata$qval          = p.adjust(indata$pval, method = "bonferroni")
    indata$type          = name
    indata               = indata[order(indata$pval),]
    
    message(paste(name, nrow(indata), nrow(indata[indata$qval < 0.05,])))
    return(indata)
}

diffexp_gene    = read_diffexp("gene_tpm"   , gene_info)
diffexp_iso_tpm = read_diffexp("isoform_tpm", isof_info)
diffexp_iso_use = read_diffexp("isoform_use", isof_info)
diffexp         = rbind(diffexp_gene, diffexp_iso_tpm, diffexp_iso_use)

fwrite(diffexp, "pipeline/4.1.differential_expression/diffexp_cell.txt", sep = "\t", col.names = TRUE, row.names = FALSE)

gene_tpm 163144 56078

isoform_tpm 306168 71475



In [ ]:
diffexp[diffexp$gene_name == "ARMCX6" & diffexp$type == "gene_tpm",]

In [ ]:
metadata   =              fread("pipeline/3.1.covariates/metadata.txt"     , sep = "\t", header = TRUE , data.table = FALSE)
covariates = add_rownames(fread("pipeline/3.1.covariates/covariates.txt"   , sep = "\t", header = TRUE , data.table = FALSE))
int_list   = readLines         ("pipeline/3.2.eqtls/vars/cardiac_eqtls.gene.txt")
int2_list  = readLines         ("pipeline/3.2.eqtls/vars/covariates_to_interaction.txt")
covariates = covariates[,unique(c(int_list, int2_list))]
gene_tpm   = add_rownames(fread("pipeline/1.2.expression/tpm_gene.normalized.txt"   , sep = "\t", header = TRUE , data.table = FALSE))
isof_tpm   = add_rownames(fread("pipeline/1.2.expression/tpm_isoform.normalized.txt", sep = "\t", header = TRUE , data.table = FALSE))
isof_use   = add_rownames(fread("pipeline/1.2.expression/use_isoform.expressed.txt" , sep = "\t", header = TRUE , data.table = FALSE))


In [ ]:
a = merge(covariates, data.frame(assay_id = colnames(gene_tpm), exp = as.numeric(gene_tpm["ENSG00000198960.11_6",])), by.x = "row.names", by.y = "assay_id")

In [ ]:
plot(a$cibersort.regular.cardiac_muscle, a$exp)